## Exploring an ItemCollection

Let's load an `ItemCollection` of Landsat 8 scenes from the Planetary Computer STAC and preview them with `stacmap.explore`.

In [1]:
import stacmap
import pystac_client

In [2]:
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

aoi = {"type": "Polygon", "coordinates": [[
    [-112.2068359375, 42.41050500280784],
    [-112.2068359375, 39.25270077697571],
    [-107.9880859375, 39.25270077697571],
    [-107.9880859375, 42.41050500280784],
    [-112.2068359375, 42.41050500280784]
]]}

items = catalog.search(
    collections=["landsat-8-c2-l2"],
    intersects=aoi,
    datetime="2019-06-01/2019-06-10"
).get_all_items()

### Footprints

Simply calling `stacmap.explore` with a STAC Collection will preview the item footprints on an interactive map. Hover over each footprint to see its STAC metadata.

In [3]:
stacmap.explore(items)

### Viewing the Search Bounds

Pass a bounding box to `bbox` or a GeoJSON to `intersects` to overlay your search bounds.

In [4]:
stacmap.explore(items, intersects=aoi)

### Custom Styling

Visualization styles can be customized using `style_kwds` and `highlight_kwds`. `stacmap.explore` follows the `geopandas.GeoDataFrame.explore` implementation for these styling options, so check out [their docs](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html#geopandas-geodataframe-explore) to get a list of supported options. You can also customize the Folium basemap using the `tiles` parameter.

In [5]:
stacmap.explore(items, style_kwds={"color": "#c44500", "weight": 4, "fillOpacity": 0.1}, highlight_kwds={"fillOpacity": 0.9}, tiles="CartoDB dark_matter")

### Customizing Tooltips and Popups

By default, `stacmap.explore` gives a tooltip that displays all STAC properties on-hover. Properties can be filtered using `fields`, and the on-hover `tooltip` can be replaced with an on-click `popup`.

In [6]:
stacmap.explore(items, tooltip=False, popup=True, fields=["eo:cloud_cover", "view:sun_azimuth", "view:sun_elevation"])

### Color-Coding

Footprints can be color-coded based on STAC properties using the `prop` parameter. This will override any `color` or `fillColor` specified in the `style_kwds`. The default colormap can be replaced with any `colorbrewer` name using `cmap`.

In [7]:
stacmap.explore(items, prop="view:sun_elevation", cmap="BrBG", style_kwds={"fillOpacity": 0.6})

### Categorical Coloring

By default, numeric properties are colored continuously and given continuous legends while non-numeric properties are treated categorically. However, you can force a numerical field to plot categorically using `force_categorical`.

In [8]:
stacmap.explore(items, prop="proj:epsg", force_categorical=True)

### Using Existing Maps

`stacmap.explore` returns a Folium map. You can add multiple STAC collections by passing an existing map to the `m` parameter. Pass a `name` parameter with each STAC collection to allow layer control. Below, we'll split the Landsat items into two sub-groups and plot each separately on the same map.

In [9]:
m = stacmap.explore(items[:5], name="Landsat #1", prop="proj:epsg", force_categorical=True)
stacmap.explore(items[5:], m=m, name="Landsat #2", prop="eo:cloud_cover", cmap="BrBG", fields=["eo:cloud_cover", "instruments"])
m

### Thumbnails

Some STAC items have links to thumbnail assets. If they are available, you can overlay thumbnail images by setting `thumbnails=True`. Note that catalogs like Planetary Computer may need to be signed to access thumbnails.

In [10]:
import planetary_computer

stacmap.explore(planetary_computer.sign(items[0]), thumbnails=True, style_kwds={"fill": False})